In [2]:
import tifffile
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
import pandas as pd
import pyvista as pv
import cv2 as cv
import numpy as np
import itertools
import scipy
import time
import pandas as pd
from sklearn.mixture import GaussianMixture as GMM
from skimage.segmentation import mark_boundaries, slic
from scipy.ndimage import generic_filter
from cv2 import fastNlMeansDenoising,fastNlMeansDenoisingMulti, medianBlur
from skimage import morphology
from skimage.measure import regionprops
import pyvista
from scipy.linalg import orthogonal_procrustes
import math

In [3]:
data = pd.read_excel('clean_data.xlsx')

In [8]:
print (data.columns)

Index(['area_x', 'area_bbox_x', 'area_convex_x', 'area_filled_x',
       'axis_major_length_x', 'axis_minor_length_x', 'bbox_x', 'centroid_x',
       'coords_x', 'equivalent_diameter_area_x', 'euler_number_x', 'extent_x',
       'inertia_tensor_x', 'solidity_x', 'label_x', 'transformed_coord',
       'matched_high_centroid', 'matched_high_label',
       'distance_transformed_high_res', 'area_y', 'area_bbox_y',
       'area_convex_y', 'area_filled_y', 'axis_major_length_y',
       'axis_minor_length_y', 'bbox_y', 'centroid_y', 'coords_y',
       'equivalent_diameter_area_y', 'euler_number_y', 'extent_y',
       'inertia_tensor_y', 'solidity_y', 'label_y', 'volume_low',
       'volume_high'],
      dtype='object')
